# Imports


In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np

# Allows for further data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt  # Plotting
import matplotlib.dates as mdates  # Styling dates

import datetime as dt

import time

import yfinance as yf

# Used for file handling like deleting files
# and get data from a directory
import os
from os import listdir
from os.path import isfile, join

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# Use plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

import warnings

warnings.simplefilter("ignore")


# Constants


In [2]:
PATH = "C:\\Users\\Eduardo Nuñez\\OneDrive\\Programming\\Python\\Derek Banas\\Investing\\Crypto\\Crypto Data\\"

# Start end date defaults
S_DATE = "2017-02-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)


# Get Dataframe from CSV


In [3]:
# A dataframe is a pandas type of object used like a spreadsheet
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df


# Get all stocks downloaded in list

In [4]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]

tickers.sort()  
len(tickers)

10

# Add daily returns

In [5]:
def add_daily_return_to_df(df: pd.DataFrame):
    df["daily_return"] = (df["Close"] / df["Close"].shift(1)) - 1
    return df

# Add Cumulative returns

In [6]:
def add_cum_return_to_df(df:pd.DataFrame):
    df["cum_return"] = (1 + df["daily_return"]).cumprod()
    return df

# Add Bollinger Bands
Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are too high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.

In [7]:
def add_bollinger_bands(df: pd.DataFrame):
    # the value 1.96 is the value for which P(X<1.96) = 0.975 and is commonly used
    # to evaluate confidence intervals

    n_days = 30  # window in days
    window = df["Close"].rolling(window=n_days)

    df["middle_band"] =window.mean()
    df["upper_band"] = df["middle_band"] + 1.96 * window.std()
    df["lower_band"] = df["middle_band"] - 1.96 * window.std()
    return df


# Add Ichimoku Data to Dataframe
The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. If you are a short term trader you create 1 minute or 6 hours. Long term traders focus on day or weekly data.

- Conversion Line (Tenkan-sen): Represents support, resistance and reversals. Used to measure short term trends.
- Baseline (Kijun-sen): Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
- Leading Span A (Senkou A): Used to identify future areas of support and resistance.
- Leading Span B (Senkou B): Other line used to identify future support and resistance.
- Lagging Span (Chikou): Shows possible support and resistance. It is used to confirm signals obtained from other lines.
- Cloud (Kumo): Space between Span A and B. Represents the divergence in price evolution.

Formulas

- Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
- Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
- Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
- Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
- Lagging Span = Price shifted back 26 periods

In [8]:
def add_ichimoku(df: pd.DataFrame):
    # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
    hi_val = df["High"].rolling(window=9).max()
    lo_val = df["Low"].rolling(window=9).min()
    df["Conversion"] = (hi_val+lo_val)/2

    # Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    hi_val2 = df["High"].rolling(window=26).max()
    lo_val2 = df["Low"].rolling(window=26).min()
    df["Baseline"] = (hi_val2+lo_val2)/2

    # Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
    df["SpanA"] = ((df["Conversion"]+df["Baseline"])/2).shift(26)
    
    # Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
    hi_val3 = df["High"].rolling(window=52).max()
    lo_val3 = df["Low"].rolling(window=52).min()
    df["SpanB"] = ((hi_val3+lo_val3)/2).shift(26)

    # Lagging Span = Price shifted back 26 periods
    df["Lagging"] = df["Close"].shift(-26)
    
    return df

# Add Daily, Cumulative Bollinger Bands & Ichimoku to Dataframes

In [9]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_stock_df_from_csv(x)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_ichimoku(new_df)
        new_df.to_csv(PATH + x + ".csv")
    except Exception as ex:
        print(ex)

print("Finished")

Working on : ADA-USD
Working on : ATOM-USD
Working on : BTC-USD
Working on : ETH-USD
Working on : ICX-USD
Working on : NEO-USD
Working on : SOL-USD
Working on : VET-USD
Working on : XTZ-USD
Working on : ZIL-USD
Finished
